In [1]:
import random
import pandas as pd
from google_play_scraper import Sort, reviews_all

s=set()
while len(s)<385:
    s.add(random.randint(1, 22687654))
idx_list=sorted(list(s))

result = reviews_all(
    'com.openai.chatgpt',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, 
    filter_score_with=None # defaults to None(means all score)
)

sample = [result[i] for i in idx_list]
print("sample size:", len(sample))
print("samples",sample[:5])

IndexError: list index out of range

In [2]:
len(result)

1188567

In [3]:
s=set()
while len(s)<385:
    s.add(random.randint(1, len(result)))
idx_list=sorted(list(s))
sample = [result[i] for i in idx_list]
print("sample size:", len(sample))
print("samples",sample[:5])

sample size: 385
samples [{'reviewId': '86d04fd4-1bdb-42a4-bf37-134dd4d44328', 'userName': 'A Google user', 'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g', 'content': 'too filtered. and too many limits for everything, fix your app openai', 'score': 1, 'thumbsUpCount': 0, 'reviewCreatedVersion': '1.2025.203', 'at': datetime.datetime(2025, 7, 31, 6, 48, 2), 'replyContent': None, 'repliedAt': None, 'appVersion': '1.2025.203'}, {'reviewId': '331345d6-69c5-4e14-8597-045e95b0a33c', 'userName': 'A Google user', 'userImage': 'https://play-lh.googleusercontent.com/EGemoI2NTXmTsBVtJqk8jxF9rh8ApRWfsIMQSt2uE4OcpQqbFu7f7NbTK05lx80nuSijCz7sc3a277R67g', 'content': 'good 👍', 'score': 5, 'thumbsUpCount': 0, 'reviewCreatedVersion': '1.2025.203', 'at': datetime.datetime(2025, 7, 30, 22, 41, 33), 'replyContent': None, 'repliedAt': None, 'appVersion': '1.2025.203'}, {'reviewId': '0f69de57-2cab-4b9c-8a9e-2bf848fc581d', 

In [4]:
df = pd.DataFrame(sample)

# 2. Save the DataFrame to a CSV file
df.to_csv('chatgpt_reviews_sample.csv', index=False)

print("Sample data saved to chatgpt_reviews_sample.csv")

Sample data saved to chatgpt_reviews_sample.csv


In [6]:
# Assume 'all_1_million_reviews' is your list containing all 1,188,567 review dictionaries

print("Converting list to DataFrame... This may take a moment.")
df = pd.DataFrame(result)

print("Saving DataFrame to CSV... This may also take some time.")
df.to_csv('chatgpt_reviews_1M_set.csv', index=False)

print("Successfully saved 1,188,567 reviews to chatgpt_reviews_1M_set.csv")

Converting list to DataFrame... This may take a moment.
Saving DataFrame to CSV... This may also take some time.
Successfully saved 1,188,567 reviews to chatgpt_reviews_1M_set.csv


In [2]:
import pandas as pd

# --- Configuration ---
# The name of your large dataset file
input_filename = 'chatgpt_reviews_1M_set.csv'
# The desired name for the new sample file
output_filename = 'chatgpt_reviews_sample.csv' # <-- FILENAME UPDATED
sample_size = 10000

df = pd.read_csv(input_filename)
sample_df = df.sample(n=sample_size, random_state=42)

# Save the new sample to the specified CSV file
sample_df.to_csv(output_filename, index=False)
print(f"\nSuccessfully saved a random sample of {len(sample_df)} reviews to '{output_filename}'.")


C:\Users\m1391\AppData\Local\Temp\ipykernel_5768\1189873596.py:10: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_filename)



Successfully saved a random sample of 10000 reviews to 'chatgpt_reviews_sample.csv'.


In [2]:
pip install google_play_scraper

  Using cached google_play_scraper-1.2.7-py3-none-any.whl.metadata (50 kB)
Using cached google_play_scraper-1.2.7-py3-none-any.whl (28 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import time
from google_play_scraper import reviews, Sort

# --- 1. Configuration ---
APP_ID = 'com.openai.chatgpt'
# Use the known total number of reviews to calculate the interval
TOTAL_REVIEWS = 22687654
SAMPLE_SIZE = 385

# Calculate the interval to get the desired sample size
INTERVAL = TOTAL_REVIEWS // SAMPLE_SIZE

# --- 2. Initialization ---
sample_reviews = []
review_count = 0
continuation_token = None # Used to get the next page of reviews
errors = 0

print(f"Starting scrape. Will sample 1 review every {INTERVAL} reviews.")

# --- 3. Main Scraping Loop ---
while len(sample_reviews) < SAMPLE_SIZE:
    try:
        # Fetch a batch of reviews
        result, continuation_token = reviews(
            APP_ID,
            lang='en',
            country='us',
            sort=Sort.NEWEST,
            count=200,  # Fetch in batches of 200
            continuation_token=continuation_token
        )

        # If no more reviews are returned, or the token is gone, stop.
        if not result or not continuation_token:
            print("Reached the end of the reviews.")
            break

        # Loop through the batch, but only save reviews at the interval
        for review in result:
            if review_count % INTERVAL == 0:
                sample_reviews.append(review)
                # Print progress
                print(f"Processed: {review_count}, Samples Found: {len(sample_reviews)}")
                # Stop once we have enough samples
                if len(sample_reviews) >= SAMPLE_SIZE:
                    break
            review_count += 1
        
        # Be respectful to Google's servers
        time.sleep(1)

    except Exception as e:
        print(f"An error occurred: {e}")
        errors += 1
        if errors > 10:
            print("Too many consecutive errors. Stopping.")
            break
        time.sleep(5) # Wait longer after an error

# --- 4. Final Result ---
print("\nScraping complete.")
df_sample = pd.DataFrame(sample_reviews)

print(f"Final Sample Size: {len(df_sample)}")
print("--- Sample DataFrame Head ---")
print(df_sample.head())

Starting scrape. Will sample 1 review every 58928 reviews.
Processed: 0, Samples Found: 1
Processed: 58928, Samples Found: 2
Processed: 117856, Samples Found: 3
Processed: 176784, Samples Found: 4
